In [12]:
import pandas as pd
from funcoes import remover_missings,escolher_estrategia_imputacao,selecao_variaveis,perfil_base,aplicar_imputacao_treino,aplicar_imputacao_teste
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from typing import List, Tuple
import joblib
import pycaret
from typing import Dict, Tuple

In [13]:
train = pd.read_csv("train.csv",sep=",")
train.shape

(6568, 81)

In [14]:
#Remove colunas que possuem um percentual de valores ausentes (missings) maior ou igual ao valor definido em perc_miss.
#Definimos que variáveis com 50% de valores ausentes serão descartadas.
#Modelos de machine learning precisam de um conjunto de dados completo e representativo. 
#Se muitas variáveis tiverem altos níveis de missings, o modelo pode:
    #Perder generalização ao imputar valores incorretos. 
    #Diminuir a capacidade preditiva, pois pode aprender padrões errados.
#

treino_pos_miss = remover_missings(train, perc_miss= 50)

Colunas removidas(38): ['VAR_10', 'VAR_12', 'VAR_14', 'VAR_16', 'VAR_18', 'VAR_21', 'VAR_23', 'VAR_26', 'VAR_27', 'VAR_29', 'VAR_31', 'VAR_36', 'VAR_37', 'VAR_41', 'VAR_42', 'VAR_43', 'VAR_46', 'VAR_47', 'VAR_48', 'VAR_49', 'VAR_50', 'VAR_51', 'VAR_55', 'VAR_56', 'VAR_61', 'VAR_62', 'VAR_63', 'VAR_66', 'VAR_67', 'VAR_68', 'VAR_69', 'VAR_70', 'VAR_71', 'VAR_73', 'VAR_74', 'VAR_75', 'VAR_77', 'VAR_78']


In [15]:
#Estratégia 
#Média: Se a coluna for numérica, sem outliers e missing < 5%.
#Mediana: Se houver outliers e missing entre 5%-20%.
#Mediana: Para outros casos de dados numéricos. (ex: sem outlier e missing alto etc..)

#Utilizar modelos para imputação (criar um modelo que faça previsões dos valores faltantes) aumenta a complexidade, deploy e avaliação.
regra_imputacao = escolher_estrategia_imputacao(treino_pos_miss)

In [16]:
#Não vamos salvar as regras imputação pois esse modelo é apenas para auxiliar na seleção de variáveis
df_imputado, regra_imputacao, dict_mediana, dict_media = aplicar_imputacao_treino(treino_pos_miss,regra_imputacao)

In [17]:
#Selecionando vari

# Classic (RFE): Remove variáveis menos importantes de forma recursiva.  
# Univariate: Seleciona variáveis com testes estatísticos (ANOVA/Qui-quadrado). Anova compara médias de variáveis e verifica se são diferentes. Qeui para variáveis categóricas
# Sequential (SFS): Adiciona ou remove variáveis uma a uma para otimizar o modelo.
#Método de combinação das variáveis selecionadas.
#intersection: Mantém apenas as variáveis escolhidas por todos os métodos.
#union: Mantém todas as variáveis selecionadas por pelo menos um método.
#voting: Mantém variáveis selecionadas por pelo menos 2 dos métodos escolhidos.

variaveis_selecionadas = selecao_variaveis(
    data=df_imputado.drop(['id','safra'],axis=1), 
    target='y', 
    methods=['classic', 'univariate', 'sequential'], 
    selection_rule='union'
)
print(variaveis_selecionadas)

[LightGBM] [Info] Number of positive: 1297, number of negative: 3300
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000434 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6692
[LightGBM] [Info] Number of data points in the train set: 4597, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.282141 -> initscore=-0.933869
[LightGBM] [Info] Start training from score -0.933869
['VAR_59', 'VAR_45', 'VAR_76', 'VAR_22', 'VAR_34', 'VAR_52', 'VAR_40', 'VAR_8', 'VAR_20', 'VAR_19', 'VAR_57', 'VAR_11', 'VAR_13', 'VAR_64', 'VAR_39', 'VAR_30', 'VAR_17', 'VAR_33', 'VAR_15', 'VAR_35', 'VAR_58', 'VAR_7', 'VAR_2', 'VAR_65', 'VAR_54', 'VAR_44', 'VAR_25', 'VAR_28', 'VAR_60', 'VAR_4', 'VAR_24', 'VAR_1', 'VAR_6', 'VAR_38', 'VAR_72', 'VAR_5', 'VAR_3', 'VAR_53', 'VAR_32', 'VAR_9']


In [18]:
#Agora vamos retornar a base de train original e Salvar a base de train_selecionada (após seleção de variáveis)
train_selecionada = train[['id','safra','y']+variaveis_selecionadas]

In [19]:
train_selecionada.head()

,id,safra,y,VAR_59,VAR_45,VAR_76,VAR_22,VAR_34,VAR_52,VAR_40,...,VAR_24,VAR_1,VAR_6,VAR_38,VAR_72,VAR_5,VAR_3,VAR_53,VAR_32,VAR_9
0,601,201409,1,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,NaN,16.0,189.0,252.00,36.0,NaN,0.0,NaN,0.057198,800.0
1,4320,201409,1,NaN,NaN,NaN,0.0,496.96,498.96,0.0,...,584.92,82.0,339.0,420.81,83.0,83.56,1.0,1928.1,0.090634,2000.0
2,8725,201406,1,NaN,NaN,NaN,0.0,135.80,NaN,0.0,...,NaN,112.0,175.0,176.19,24.0,NaN,1.0,500.0,0.057198,500.0
3,8977,201402,0,331.16,159.95,377.95,50.0,45.99,39.99,11.0,...,306.02,0.0,3069.0,NaN,43.0,67.66,0.0,2000.0,0.133833,1000.0
4,3547,201402,0,NaN,NaN,NaN,0.0,157.96,NaN,0.0,...,200.55,20.0,438.0,127.76,59.0,40.11,0.0,600.0,0.090634,300.0


In [20]:
train_selecionada.shape

(6568, 43)

In [21]:
#Verificando nossa base após seleção de variáveis
resultado = perfil_base(train_selecionada, id_col='id', target_col='y', safra_col='safra')

Calcula métricas básicas do perfil da base de dados.
Shape da base: Essa base possui 6568 linhas e 43 colunas
Tipos de variáveis: {dtype('float64'): 37, dtype('int64'): 6}
IDs únicos: 6568
Taxa de maus (bad rate): Bons: 4715(71.8 %), Maus: 1853 (28.2%)
Volumetria das safras: {201401: 702, 201402: 709, 201403: 701, 201404: 754, 201405: 772, 201406: 715, 201407: 777, 201408: 727, 201409: 711}




In [22]:
#Salvando a base de treino_selecionada
train_selecionada.to_csv("train_selecionada.csv",sep=",",index=False,header=True)